# Evaluation
## Setup

In [ ]:
import plotly.express as px
import pandas as pd

import aggregation.aggregator as ag
import data.file_handler as fh
import data.file_name_generator as fg
from aggregation.pipeline_blocks import add_sense_counts_to_id_map, calc_ari, calc_ari_per_token, calc_silhouette_score_per_token
from clustering.linkage_name import LinkageName
from clustering.metric_name import MetricName
from data.corpus_handler import CorpusName, CorpusHandler

In [ ]:
CORPUS_NAMES = [CorpusName.SEMCOR.value, CorpusName.SENSEVAL2.value,
                CorpusName.SENSEVAL3.value, CorpusName.SEMEVAL07.value,
                CorpusName.SEMEVAL13.value, CorpusName.SEMEVAL15.value]
CORPUS_CACHE_PATH = './data/corpus_cache'
RESULTS_PATH = fh.add_and_get_abs_path('./data/results')

In [ ]:
SHOW_FIG = False
SAVE_FIG = False

In [ ]:
LABEL_DICT = {'unique_sense_count': 'Unique Sense Count',
              'total_token_count': 'Total Token Count',
              'unique_token_count': 'Unique Token Count',
              'ari': 'ARI',
              'silhouette_score': 'Silhouette Score'}
WIDTH = 512
HEIGHT = 256

## Corpus Evaluation
### Corpus Statistics as LaTeX Table

In [ ]:
for corpus_name in CORPUS_NAMES:
    corpus = CorpusHandler(corpus_name, CORPUS_CACHE_PATH)
    st = ag.calc_corpus_statistics_for_tagged_senses(corpus)
    print(f"    \\textbf{{{corpus.corpus_name}}} & ${st['unique_sense_count']:,}$ & ${st['total_sense_count']:,}$ & ${st['unique_token_count']:,}$ & ${st['total_token_count']:,}$ & ${st['unique_monosemous_token_count']:,}$ & ${st['total_monosemous_token_count']:,}$ & ${st['unique_polysemous_token_count']:,}$ & ${st['total_polysemous_token_count']:,}$ \\\\")

### Our Baselines as LaTeX Table

Shows the No Disambiguation baseline (one cluster/sense per unique token) and Complete Disambiguation baseline (one cluster/sense per token) per corpus.

In [ ]:
for corpus_name in CORPUS_NAMES:
    corpus = CorpusHandler(corpus_name, CORPUS_CACHE_PATH)
    tagged_tokens = corpus.get_tagged_tokens()

    tagged_tokens['sense'] = range(tagged_tokens.token.count())
    ari_no_disambiguation = calc_ari(corpus.get_tagged_tokens(), tagged_tokens)['ari']

    tagged_tokens['sense'] = tagged_tokens.token.factorize()[0]
    ari_complete_disambiguation = calc_ari(corpus.get_tagged_tokens(), tagged_tokens)['ari']

    print(f"    \\textbf{{{corpus_name}}} & ${ari_no_disambiguation:.4f}$ & ${ari_complete_disambiguation:.4f}$ \\\\")

### Relationship Between Sense Counts and Token Counts as Scatter Plot

In [ ]:
for corpus_name in CORPUS_NAMES:
    corpus = CorpusHandler(corpus_name, CORPUS_CACHE_PATH)
    tagged_tokens = corpus.get_tagged_tokens()
    tagged_tokens_and_counts = ag.count_unique_senses_per_token(tagged_tokens[tagged_tokens.tagged_sense])
    token_counts_per_sense_count = ag.count_tokens_per_sense_count(tagged_tokens_and_counts)

    fig = px.scatter(token_counts_per_sense_count, x='unique_sense_count', y='unique_token_count',
                     color='total_token_count', size='total_token_count', color_continuous_scale=px.colors.sequential.Greys,
                     log_x=True, log_y=True, labels=LABEL_DICT,
                     template='plotly_white', width=WIDTH, height=WIDTH)
    fig.update_layout(coloraxis_colorbar=dict(title_side='right',
                                              thicknessmode="fraction", thickness=0.04,
                                              ticks="outside"))
    fig.update_traces(marker={'line': {'color': 'black'}})

    if SHOW_FIG:
        fig.show()
    if SAVE_FIG:
        fig.write_image(f'data/plots/fig_{corpus_name}_tokens_and_senses.pdf')

## Dictionary Evaluation
### ARI per Unique Sense Count as Bar Plot

In [ ]:
for corpus_name in CORPUS_NAMES:
    sense_stats = list()
    for linkage_name in LinkageName.get_values():
        corpus = CorpusHandler(corpus_name, CORPUS_CACHE_PATH)
        experiment_prefix = fg.gen_experiment_prefix_no_dist(corpus_name, MetricName.EUCLIDEAN, linkage_name)
        dictionary = fh.load_df(RESULTS_PATH, fg.gen_dictionary_file_name(experiment_prefix))
        dictionary = calc_ari_per_token(corpus.get_tagged_tokens(), dictionary)
        if 'unique_sense_count' not in dictionary.columns:
            dictionary = add_sense_counts_to_id_map(corpus.get_tagged_tokens(), dictionary)
        ari_per_sense_count = dictionary[dictionary.tagged_token].groupby(by='unique_sense_count').aggregate({'ari': 'mean'}).reset_index()
        ari_per_sense_count['Linkage'] = linkage_name
        sense_stats.append(ari_per_sense_count)

    ari_per_sense_count = pd.concat(sense_stats)
    fig_ari_per_sense_count = px.bar(
        ari_per_sense_count, x='unique_sense_count', y='ari',
        pattern_shape='Linkage', color='Linkage', barmode='group',
        labels=LABEL_DICT, template='plotly_white', width=WIDTH, height=HEIGHT)
    fig_ari_per_sense_count.update_xaxes(
        showgrid=False, ticks="outside", tickson="labels", ticklen=4,
        nticks=int(ari_per_sense_count.unique_sense_count.max()))

    if SHOW_FIG:
        ari_per_sense_count.show()
    if SAVE_FIG:
        fig_name = f'fig_{corpus_name}-affinity_euclidean-ari-hist'
        fig_ari_per_sense_count.write_image(f'data/plots/{fig_name}.pdf')
        print(f"  \subfloat[][{corpus_name}]{{\includegraphics[width=0.49\\textwidth]{{./fig/{fig_name}}}}} \\;")

### Silhouette Coefficient per Unique Sense Count as Bar Plot

In [ ]:
for corpus_name in CORPUS_NAMES:
    sense_stats = list()
    for linkage_name in LinkageName.get_values():
        corpus = CorpusHandler(corpus_name, CORPUS_CACHE_PATH)
        experiment_prefix = fg.gen_experiment_prefix_no_dist(corpus_name, MetricName.EUCLIDEAN, linkage_name)
        word_vectors = fh.load_matrix(RESULTS_PATH, fg.gen_word_vec_file_name(corpus.corpus_name))
        dictionary = fh.load_df(RESULTS_PATH, fg.gen_dictionary_file_name(experiment_prefix))
        dictionary = calc_silhouette_score_per_token(word_vectors, dictionary, MetricName.EUCLIDEAN)
        dictionary = calc_ari_per_token(corpus.get_tagged_tokens(), dictionary)
        if 'unique_sense_count' not in dictionary.columns:
            dictionary = add_sense_counts_to_id_map(corpus.get_tagged_tokens(), dictionary)
        silhouette_score_per_sense_count = dictionary[dictionary.tagged_token].groupby(by='unique_sense_count').aggregate({'silhouette_score': 'mean'}).dropna().reset_index()
        silhouette_score_per_sense_count['Linkage'] = linkage_name
        sense_stats.append(silhouette_score_per_sense_count)

    silhouette_score_per_sense_count = pd.concat(sense_stats)
    fig_silhouette_score_per_sense_count = px.bar(
        silhouette_score_per_sense_count, x='unique_sense_count', y='silhouette_score',
        pattern_shape='Linkage', color='Linkage', barmode='group',
        labels=LABEL_DICT, template='plotly_white', width=WIDTH, height=HEIGHT)
    fig_silhouette_score_per_sense_count.update_xaxes(
        showgrid=False, ticks="outside", tickson="labels", ticklen=4,
        nticks=int(silhouette_score_per_sense_count.unique_sense_count.max()))
    if SHOW_FIG:
        fig_silhouette_score_per_sense_count.show()
    if SAVE_FIG:
        fig_name = f'fig_{corpus_name}-affinity_euclidean-silhouette-hist'
        fig_silhouette_score_per_sense_count.write_image(f'data/plots/{fig_name}.pdf')
        print(f"  \subfloat[][{corpus_name}]{{\includegraphics[width=0.49\\textwidth]{{./fig/{fig_name}}}}} \\;")

### Browse a Dictionary as DataFrame

In [ ]:
#EXPERIMENT_PREFIX = fg.gen_experiment_prefix(corpus.corpus_name, MetricName.COSINE, LinkageName.SINGLE, 0.4)
#EXPERIMENT_PREFIX = fg.gen_experiment_prefix_no_dist(corpus.corpus_name, MetricName.EUCLIDEAN, LinkageName.SINGLE)
dictionary = fh.load_df(RESULTS_PATH, fg.gen_dictionary_file_name(EXPERIMENT_PREFIX))
ag.pack_sentence_ids_and_token_ids(
    ag.unpack_and_sort_per_token_id(
        dictionary,
        ['sentence_id', 'token_id', 'sense']),
    ['token', 'sense'])